In [4]:
import pandas as pd
import os
import numpy as np
import json
import jellyfish

In this notebook we will create a scores matrix as well as group the scores for levenshtein similarity.
We will also turn these groups into a dataframe that can be used later in the pipeline in the notebook "new figures"

In [5]:
#similarity scores for tools
with open('/home/marinedjaffardjy/Documents/Code/Similarite_process/json/levenshtein_nf_tools/levenshtein_nf_tools_3800.json') as f1:
    scores = json.load(f1)
#snakemake rules
with open('/home/marinedjaffardjy/Documents/Code/Investigating_reuse/json/snk_rule_info_tool.json') as f:
    snk_rules = json.load(f)

In [12]:
def grouping_simple(matrice_sim,obj_liste,treshold=0.85):
    to_ignore=[]
    groups=[]
    i=0
    for i in range(0,len(matrice_sim)):
        if i not in to_ignore:
            new_group=[i]
            for j in range(0,len(matrice_sim[i])):
                if(matrice_sim[i][j]>treshold):
                    new_group.append(j)
            groups.append(list(set(new_group)))
            to_ignore+=new_group
            
    #translate the groups into snk rules
    simil_obj=[]
    for group in groups:
        group_obj = []
        for el in group:
            group_obj.append(obj_liste[el])
        simil_obj.append(group_obj)
    return simil_obj
    
        
        
        

In [13]:
%%time
group_snk = grouping_simple(matrix_scores,snk_rules)

CPU times: user 2.37 s, sys: 123 µs, total: 2.37 s
Wall time: 2.37 s


In [14]:
with open("/home/marinedjaffardjy/Documents/Code/Similarite_process/json/simil_groups_levenshtein_tools.json","w") as f:
    json.dump(group_snk,f)

In [27]:
def grouping_old(matrix_scores, treshold, snk_rules):
    #input
    #output
    #getting the similarity groups per line
    tab_sim = []
    for line in matrix_scores:
        simil = []
        for i in range(0,len(line)):
            if(line[i]>=treshold):
                simil.append(i)
            i+=1
        tab_sim.append(simil)
    #transitivity
    #grouping the similarity groups by parsing the matrix x times (x = nb of processes)
    #TODO : optimiser
    
    for i in range(len(snk_rules)-1,-1,-1):
        new_line = []
        to_pop = []
        for line_nb in range(0,len(tab_sim)):
            if i in tab_sim[line_nb]:
                new_line = new_line+tab_sim[line_nb]
                to_pop.append(line_nb)
        to_pop.sort(reverse=True)
        for p in to_pop:
            tab_sim.pop(p)
        tab_sim.append(list(set(new_line)))
            
    #translate the groups into snk rules
    simil_snk=[]
    for group in tab_sim:
        group_snk = []
        for el in group:
            group_snk.append(snk_rules[el])
        simil_snk.append(group_snk)
    return simil_snk
        
        

In [29]:
%%time
group_snk_old = grouping_old(matrix_scores,0.85,snk_rules)
with open("/home/marinedjaffardjy/Documents/Code/Similarite_process/json/simil_groups_levenshtein_tools_old.json","w") as f:
    json.dump(group_snk_old,f)

CPU times: user 6.11 s, sys: 62.6 ms, total: 6.17 s
Wall time: 6.19 s


In [35]:
def importing_json_files(file_wf):
    f_wf = open(file_wf) #informations for nf
    # returns JSON object as
    # a dictionary
    wf = json.load(f_wf)
    f_wf.close
    return wf

#importing the wf and auth dict (github info)
dict_nf = importing_json_files('json/wf_new_crawl_nextflow.json')
auth_nf = importing_json_files('json/author_clem_nf.json')
dict_snk = importing_json_files('json/wf_crawl_snakemake.json')
auth_snk = importing_json_files('json/author_clem_snk.json')



In [5]:
with open("/home/marinedjaffardjy/Documents/Code/Similarite_process/json/simil_groups_levenshtein_tools.json") as f:
    group_snk=json.load(f)

In [15]:
len(group_snk)

3724

In [16]:
#if the groups are ill-formed
group_snk = [x for x in group_snk if x != []]


In [17]:
len(group_snk)

3724

In [19]:
els_groups_new=[len(x) for x in group_snk]
sum(els_groups_new)

8670

In [33]:
def sim_to_df_snk(groups_list, dict_wf): 
    groups_sim_df = pd.DataFrame(columns = ['nb_reuse', 'tools','nb_own', 'list_own', 'nb_contrib','list_contrib','nb_wf','list_wf',"list_proc"])
    for group in groups_list :
        list_own = []
        list_wf = []
        list_contrib = []
        for el in group :
            list_own.append(el['owner'])
            list_wf.append(el['wf_orig'])
            name_wf = el['wf_orig'].split('/')[0]+'/'+el['wf_orig'].split('/')[1]
            for contr in dict_wf[name_wf]['contributors']:
                list_contrib.append(contr)
        list_own = list(set(list_own))
        list_wf = list(set(list_wf))
        list_contrib = list(set(list_contrib))
        groups_sim_df = groups_sim_df.append({'nb_reuse' : len(group),
                                             'nb_own' : len(list_own),
                                             'tools' : group[0]['tools'],
                                             'list_own' : list_own ,
                                             'nb_wf' : len(list_wf),
                                             'list_wf' : list_wf,
                                             'list_contrib' : list_contrib,
                                             'nb_contrib' :len(list_contrib),
                                             "list_proc": group}, ignore_index = True)
    return groups_sim_df

In [36]:
%%time
df_sim_snk = sim_to_df_snk(group_snk, dict_snk)

CPU times: user 18.7 s, sys: 64.3 ms, total: 18.8 s
Wall time: 18.8 s


In [16]:
len(df_sim_snk)

2233

In [37]:
#save the files
df_sim_snk.to_csv('csv/df_groups_lev_tools_snk.csv')

In [28]:
df_sim_snk_ord

,nb_reuse,tools,nb_own,list_own,nb_contrib,list_contrib,nb_wf,list_wf,list_proc
2176,3414,[FLASH],410,"[michalogit, chuckzzzz, pawel125, supernifty, ...",549,"[michalogit, w-bonelli, jvanheld, chuckzzzz, b...",912,"[itatop/popsimenv/3, wfondrie/snakemake-beer-p...","[{'name': 'Merge_Pairs_With_Flash', 'code': '(..."
2080,8,"[SAMtools, SAMtools]",1,[CGI-stLFR],1,[Ellis-Anderson],2,"[CGI-stLFR/CGI_de_novo_sim_Pipeline/1, CGI-stL...","[{'name': 'run_minimap_binned_contigs_refA', '..."
1933,8,[SAMtools],1,[GeorgiaBreckell],1,[GeorgiaBreckell],1,[GeorgiaBreckell/assembly_pipeline/1],"[{'name': 'samtools_coverage_nanopore', 'code'..."
1809,6,[MultiQC],2,"[LoGT-KULeuven, LUMC]",4,"[Redmar-van-den-Berg, rhpvorderman, 0mician, s...",3,"[LoGT-KULeuven/y2h_Bam35-Bt_analysis/2, LoGT-K...","[{'name': 'multiqc', 'code': '(name='multiqc',..."
1275,6,[Snakemake],3,"[dmvandenberg, ares-lab, arielgalindoalbarran]",7,"[dmvandenberg, amedina-liigh, jaimicore, jvanh...",4,"[arielgalindoalbarran/Epromoters/4, dmvandenbe...","[{'name': 'dag', 'code': '(name='dag', lineno=..."
...,...,...,...,...,...,...,...,...,...
2089,1,[SAMtools],1,[CGI-stLFR],2,"[arcadianlyric, Ellis-Anderson]",1,[CGI-stLFR/CGI_WGS_Pipeline/3],"[{'name': 'run_flagstat', 'code': '(name='run_..."
2158,1,[Bowtie],1,[AlexanderMoerseburg],1,[SherineAwad],1,[AlexanderMoerseburg/ATAC-Seq/1],"[{'name': 'tosam', 'code': '(name='tosam', lin..."
2090,1,[SAMtools],1,[CGI-stLFR],2,"[arcadianlyric, Ellis-Anderson]",1,[CGI-stLFR/CGI_WGS_Pipeline/3],"[{'name': 'duplicate_analysis', 'code': '(name..."
2094,1,[Bowtie],1,[CFarcy],1,[CFarcy],1,[CFarcy/snakemake_for_tassel/1],"[{'name': 'bowtie2', 'code': '(name='bowtie2',..."


In [24]:
df_sim_snk_ord = df_sim_snk.sort_values(by=["nb_reuse"], ascending=False)
names_and_wf = [el["wf_orig"]+"/"+el["name"] for el in df_sim_snk_ord["list_proc"][0]]

In [25]:
names_and_wf

['zavolanlab/polyAsite_workflow/1/fq2fasta_se']

Other versions of grouping

In [ ]:
def grouping_old_version(matrix_scores, snk_rules, thold, proc_names):
    groups = []
    rules = snk_rules.copy()
    tested = []
    for el in rules:
        group = []
        name_el1 = el["wf_orig"]+"/"+el["name"]
        idx1 = proc_names.index(name_el1)
        group = [el]
        to_test = [idx1]
        while len(to_test)>0:
            testing = to_test[0]
            tested = tested+[testing]
            print(testing)
            idx_n = [idx for idx, element in enumerate(matrix_scores[testing-1]) if element > thold]
            new_to_test = []
            for new_id in idx_n:
                if(new_id not in tested):
                    new_to_test.append(new_id)
            to_test=list(set(to_test[1:]+new_to_test))
            for test in to_test:
                for ru in rules:
                    wf_or = proc_names[test].replace(proc_names[test].split("/")[-1],'')
                    n = proc_names[test].split("/")[-1]
                    if(ru["wf_orig"]== wf_or and ru["name"]== n):
                        group.append(ru)
                        rules.remove(ru)
        groups.append(group)
    return(groups)

In [32]:
def grouping_simple(matrix_scores, treshold, snk_rules):
    tab_sim = []
    for line in matrix_scores:
        simil = []
        for i in range(0,len(line)):
            if(line[i]>=treshold):
                simil.append(i)
            i+=1
        tab_sim.append(simil)
    #translate the groups into snk rules
    simil_snk=[]
    for group in tab_sim:
        group_snk = []
        for el in group:
            group_snk.append(snk_rules[el])
        simil_snk.append(group)
    return simil_snk
        
    